Import necessary libraries and files.

In [3]:
import sys
import argparse
import numpy as np
import time
import zipfile
import math
import textwrap
import pprint as pprint
import random

random.seed(1)

from collections import defaultdict, Counter, OrderedDict

test_raw = '/content/test.fasta'
bound_raw = '/content/bound.fasta'
notbound_raw = '/content/notbound.fasta'

File processing: format the fasta files.

In [4]:
# build array of sequences -> later, reconstruct seq name by just returning index
def file(fname):

  # new array element for each newline
  contents = []
  with open(fname, "r") as fh:
    for line in fh:
      contents.append(line) 

  return [''.join([segment[:-1] for segment in contents[5 * i + 1:5 * i + 5]]) for i in range(len(contents) // 5)]

Add pseudocounts to motifs.

In [5]:
def ProfileWithPseudocounts(Motifs):
    count = CountWithPseudocounts(Motifs)
    profile = {}
    for nucleotide in count:
        profile[nucleotide] = [
            float(count[nucleotide][i]) / float(sum(count[nucleotide]))
            for i in range(len(count[nucleotide]))
        ]
    return profile

def CountWithPseudocounts(Motifs):
    count = {}
    k = len(Motifs[0])
    for nucleotide in "ACGT":
        count[nucleotide] = [1] * k
    for i in range(len(Motifs)):
        for j in range(k):
            nucleotide = Motifs[i][j]
            count[nucleotide][j] += 1
    return count

Helper functions for Gibbs.

In [6]:
def HammingDistance(p, q):
    return sum([1 for i in range(len(p)) if p[i] != q[i]])


def Normalize(probabilities):
    factor = 1.0 / sum(probabilities.values())
    for key in probabilities:
        probabilities[key] *= factor
    return probabilities


def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p

def randomMotif(seq, k):
    i = random.randint(0, len(seq) - k)
    return seq[i : i + k]

def WeightedDie(probabilities):
    kmer = ""
    r = random.uniform(0, 1)
    for key in probabilities:
        r -= probabilities[key]
        if r < 0:
            kmer = key
            break
    return kmer

def Score(Motifs):
    consensus = Consensus(Motifs)
    score = 0
    for motif in Motifs:
        score += HammingDistance(motif, consensus)
    return score


def Consensus(Motifs):
    count = CountWithPseudocounts(Motifs)
    k = len(Motifs[0])
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

Build PWM using Gibbs sampling.

In [7]:
def ProfileGeneratedString(Text, profile, k):
    n = len(Text)
    probabilities = {}
    for i in range(n - k + 1):
        probabilities[Text[i : i + k]] = Pr(Text[i : i + k], profile)
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)

def GibbsSampler(Dna, k, t, N):
    BestMotifs = None
    BestScore = float("inf")
    for start in range(20):
        Motifs = [randomMotif(seq, k) for seq in Dna]
        for j in range(N):
            i = random.randint(0, t - 1)
            MotifI = Motifs[i]
            MotifsWithoutI = Motifs[:i] + Motifs[i + 1 :]
            profile = ProfileWithPseudocounts(MotifsWithoutI)
            MotifI = ProfileGeneratedString(Dna[i], profile, k)
            Motifs = MotifsWithoutI[:i] + [MotifI] + MotifsWithoutI[i:]
            if Score(Motifs) < BestScore:
                BestMotifs = Motifs
                BestScore = Score(Motifs)
                print(profile)
    return BestMotifs

bound_processed = file("/content/bound.fasta")
k = 21 #length of motif we're currently looking at. ITERATE OVER THIS LATER.
t = len(bound_processed)
N = 1 #1000 is worse than 100

best_motifs = GibbsSampler(bound_processed, k, t, N)
# print(best_motifs)
# print(" ".join(best_motifs))

{'A': [0.04606772149438246, 0.04820588578315126, 0.04843914006919877, 0.04626210006608871, 0.04766162578237375, 0.046728608638183726, 0.04758387435369125, 0.04696186292423123, 0.0494499086420713, 0.04832251292617502, 0.04871127006958753, 0.04871127006958753, 0.047894880068421256, 0.046145472923064965, 0.04723399292461999, 0.045795591493993704, 0.04750612292500875, 0.05077168292967383, 0.04793375578276251, 0.04641760292345372, 0.04719511721027874], 'C': [0.05009269693554837, 0.04732996473881275, 0.049511069104656656, 0.04987458649896397, 0.04649387473190592, 0.04765713039368934, 0.048747682576611294, 0.048020647787996654, 0.04718455778108983, 0.04656657821076739, 0.04787524083027373, 0.04714820604165909, 0.04605765385873714, 0.04696644734450543, 0.04645752299247519, 0.04765713039368934, 0.04769348213312007, 0.04602130211930641, 0.046748336907921045, 0.048602275618888365, 0.04729361299938202], 'G': [0.04759561304836896, 0.04664651293588302, 0.046787120359955005, 0.047068335208098985, 0.0

In [13]:
#Build PWM from best_motifs
pwm = {'A': [0] * k,'C': [0] * k,'G': [0] * k,'T': [0] * k}

saved_best_motifs = (0, [0.2086499123319696, 0.13247613481394896, 0.21663744398986948, 0.18624586012078706, 0.10403272939801286, 0.15702318332359244, 0.3159945451003312, 0.04149620105201637, 0.14650301967660237, 0.03389830508474576, 0.11007208260276641, 0.0676017923241769, 0.22852133255406196, 0.15819209039548024, 0.049678550555230856, 0.3154100915643873, 0.11669588934346387, 0.3904149620105202, 0.1542957334891876, 0.20806545879602573, 0.20572764465225016, 0.3438534969803234, 0.586791350087668, 0.45314630820183127, 0.14825638028443405, 0.023183323592441067, 0.17221897525813365, 0.364114552893045, 0.9210987726475746, 0.192090395480226, 0.27586206896551724, 0.1441652055328268, 0.20183128774595752, 0.22969023962594973, 0.4172998246639392, 0.8702513150204558, 0.2579388271965712, 0.46425092538476526, 0.14241184492499512, 0.1636469900642899, 0.2649522696278979, 0.2943697642704072, 0.23982076758231055, 0.13189168127800507, 0.06935515293200857, 0.15254237288135594, 0.632768361581921, 0.561465030196766, 0.18293395675043833, 0.017923241768946035, 0.004480810442236509, 0.24995129553867135, 0.16578998636275083, 0.5895187999220729, 0.22423533995714007, 0.21137736216637443, 0.020455873758036237, 0.010714981492304695, 0.2857977790765634, 0.154100915643873, 0.5172413793103449, 0.32924215858172606, 0.27274498344048315, 0.20767582310539645, 0.14884083382037794, 0.26086109487629067, 0.512955386713423, 0.24001558542762516, 0.10929281122150789, 0.13695694525618546, 0.01948178453146308, 0.6569257744009351, 0.44028833041106563, 0.5799727255016559, 0.1410481200077927, 0.31755308786284825, 0.2131307227742061, 0.05961426066627703, 0.4159360997467368, 0.13325540619520748, 0.3130722774206117, 0.16481589713617767, 0.1977401129943503, 0.22715760763685955])
saved_pwm = {'A': saved_best_motifs[1][:k], 'C': saved_best_motifs[1][k:2*k], 'G': saved_best_motifs[1][2*k:3*k], 'T': saved_best_motifs[1][3*k:4*k]}

for i in range(k): #from 0 to 21, or whatever we decide K to be
  #access i-th element of each element 
  for sequence in best_motifs:
    if sequence[i] == 'A':
      pwm['A'][i] += 1
    elif sequence[i] == 'C':
      pwm['C'][i] += 1
    elif sequence[i] == 'G':
      pwm['G'][i] += 1
    elif sequence[i] == 'T':
      pwm['T'][i] += 1
 
# then divide every element in PWM by # sequences
for element in pwm:
  for q in range(k):
    pwm[element][q] = pwm[element][q] / len(best_motifs)

print(saved_pwm)

{'A': [0.2086499123319696, 0.13247613481394896, 0.21663744398986948, 0.18624586012078706, 0.10403272939801286, 0.15702318332359244, 0.3159945451003312, 0.04149620105201637, 0.14650301967660237, 0.03389830508474576, 0.11007208260276641, 0.0676017923241769, 0.22852133255406196, 0.15819209039548024, 0.049678550555230856, 0.3154100915643873, 0.11669588934346387, 0.3904149620105202, 0.1542957334891876, 0.20806545879602573, 0.20572764465225016], 'C': [0.3438534969803234, 0.586791350087668, 0.45314630820183127, 0.14825638028443405, 0.023183323592441067, 0.17221897525813365, 0.364114552893045, 0.9210987726475746, 0.192090395480226, 0.27586206896551724, 0.1441652055328268, 0.20183128774595752, 0.22969023962594973, 0.4172998246639392, 0.8702513150204558, 0.2579388271965712, 0.46425092538476526, 0.14241184492499512, 0.1636469900642899, 0.2649522696278979, 0.2943697642704072], 'G': [0.23982076758231055, 0.13189168127800507, 0.06935515293200857, 0.15254237288135594, 0.632768361581921, 0.56146503019

Score test string against PWM.

In [14]:
len_pwm = 21 #length of pwm = length of bound transcription factor

def score_string(test_string):
  # score each index of test string against each index of pwm
  score = 1
  for i in range(len(test_string)): 
    score *= saved_pwm[test_string[i]][i] # value
  return score

Iterate through the test file sequences, score each sequence against the PWM.

In [15]:
def iterate():
  test_processed = file(test_raw)
  total_scores = [['seq' + str(x + 1), 0] for x in range(len(test_processed))] #array of [seq_index, score]
  for k in range(len(test_processed)):
    max_score = 0
    for i in range(len(test_processed[k]) - len_pwm + 1):
      curr_score = score_string(test_processed[k][i:i+len_pwm])
      if curr_score > max_score: max_score = curr_score
    total_scores[k][1] = max_score
  return total_scores

Sort the sequence + score array in order of best scores to last.

In [ ]:
# MAIN FUNCTION: output top 2000
def output2000():
  total_scores = iterate()
  total_scores_sorted = sorted(total_scores, key = lambda x: x[1], reverse = True)
  # return(total_scores_sorted[0:1999])
  for i in range(0,2000): print(total_scores_sorted[i][0])
  pass

# 557 correct with random seed = 1, gibbs iterations N = 10

output2000()

Currently at 557. Pretty fast function.
Things I could do to improve it tomorrow:

- Play around with number of times we run Gibbs. Currently I only do 100. Maybe 1000?
- Loop through a variety of values for k. Currently at 21. 
- Work with reverse
- Expec max or ML
- Incorporate notbound.fasta somehow... i.e learn what the pwm looks like for the NOT bound files. Then, if a test sequence matches well with the notbound pwm, it's probably not bound by a tf. PROBLEM: the pwm is generated by looking for the "most similar thing", which means that we're implicitly assuming there is a transcription factor bound. 

Update 5/10/23: ran for 50,000 iterations and finally got 639.